# <font color='navy'><center>Evaluación Data Science</center></font>

Se le asigna como tarea realizar un modelo que determine si un usuario de MercadoPago (MP) Argentina volverá a transaccionar en los próximos 2 meses. Para ello se cuenta con los siguientes archivos:

* **PAYMENTS.csv:** Contiene datos transaccionales de MP que se hayan realizado en los últimos 6 meses. Este está sólo limitado al segmento Wallet (todos los pagos que se realicen con billetera virtual).
    * FECHA: Fecha en que se realizó exitosamente la transacción.
    * CUS_CUST_ID_SEL: ID del seller.
    * CUS_CUST_ID_BUY: ID del buyer.
    * SPENT: Dinero gastado en la transacción.
    * TPV_SEGMENT_DETAIL: Flujo que usó el usuario (Instore = QR, Transport = SUBE, Utilities = Pago de servicios, money transfer, Cellphone recharge).
    * DESCUENTO: Descuento que MP le otorgó en esa transacción.


* **ACTIVE_USER.csv:** Contiene datos de las visitas a la APP.
    * CUS_CUST_ID_BUY: ID del buyer.
    * MAU_MP_1: Cantidad de veces el usuario entró a la APP  de MP (azul) en el último mes de entrenamiento (mes -1).
    * MAU_ML_1: Cantidad de veces el usuario entró a la APP  de ML (amarilla) en el último mes de entrenamiento (mes -1).
    * MAU_ML_2: Cantidad de veces el usuario entró a la APP  de ML (amarilla) en el mes -2 de entrenamiento.
    * last_login_mp_date_1: Fecha del último login del mes en APP MP.


* **DEMOGRAFICOS.csv:** Contiene datos de la edad, sexo, tarjeta que usa y lugar de residencia.


* **DINERO_CUENTA.csv:** Contiene datos de la cantidad de dinero en la cuenta y si invierte en fondo.
    * PLATA_CUENTA_1: Máximo dinero en cuenta en el último mes (mes -1).
    * PLATA_CUENTA_2: Máximo dinero en cuenta en el mes -2.
    * INVERSION: Indica en que estado esta el usuario para la inversión:
        * Investing: Está invirtiendo.
        * Elegible: Está disponible para que inviertan dinero.
        * Warmup: Hicieron el onboarding de inversión pero no llenaron los campos.
        * Pending: Hicieron el proceso para invertir, pero está en pending de aprobación.


* **MARKETPLACE_DATA.csv:** Contiene un resumen a nivel usuario de movimientos en mercadolibre (ML) para los últimos 6 meses.
    * SPENT_ML: Dinero gastado en ML.
    * RECENCY_ML: Fecha de la ultimo compra en ML.
    * FREQUENCY_ML: Cantidad de días que realizó compras.

### Consideraciones para la entrega

Todos los datasets contienen el **CUS_CUST_ID_BUY** mediante el cual se joinean.
Para entrenar, use el archivo **EVALUATE.csv.** Tome el criterio que crea conveniente para entrenar y evaluar el modelo.

***
¿Qué información extra consideraría o qué features generaría, si pudiera, para mejorar el modelo?
***

A continuacion presento un breve  resumen de las 2 notebooks en las cuales trabaje para este desafio.
En la primera notebook revise los datos, los limpie y agrege features e informacion de relevancia.
En la segunda notebook entrene el modelo.

# Notebook 1_ProcessData

### Data Demographics

En este dataset use un par de regex para limpiar los rangos etarios, asi como unir caba y capital federal en al feature CITY. Rellene los pocos nulls que habia de distintas formas de aceurdo a la feature.

<img src="img/capture 1.PNG" alt="Drawing"/>

<img src="img/capture 2.PNG" alt="Drawing"/>

<img src="img/capture 3.PNG" alt="Drawing"/>

### Payments Dataset

De este dataset saque un par de features interesantes. El total que gasto en los ultimos 7 meses, el total de transaciones, cuantas transacciones hizo por mes, 
el promedio de descuento que obtuve en sus compras. Algo interesante es que la mayoria de gente le compro a los mismos 4 sellers, entonces puse como feature si le 
habian comprado a alguno de estos 4. Tambien saque 4 columnas correspondientes a los tipos de gastos que hacen, ya que peude ser que los tipos de gastos esten asociados
a al concurrencia de los clientes. Algo interesante seria añadir dias de descuentos masivos al estilo Black Friday, o dias festivos que la gente suele regalar cosas. 
Puede ser que una persona paresca que se va a ir de la aplicacion, pero en realidad esta persona solo gasta en dias de descuento y dias festivos, entonces la epoca en la cual
se adquiere la data de dicha persona afecta mucho

<img src="img/capture 4.PNG" alt="Drawing"/>

<img src="img/capture 5.PNG" alt="Drawing"/>

In [6]:
#el x_trans es la cantidad de transacciones que realizo ese mes
# el sell_id_xxx es si el tipo le compro o no a los 4 que mas le compra la gente

### MarketPlace y Active User

estos dos datasets fueron mas de rellenar datos faltantes con distintas tecnicas. Dependiendo el caso se uso la media, la moda o la mediana. En algunos casos como las fechas faltantes samplee de una distribución usando la priori de lso datos.

<img src="img/capture 6.PNG" alt="Drawing"/>

<img src="img/capture 6.5.PNG" alt="Drawing"/>

<img src="img/capture 7.PNG" alt="Drawing"/>

### Por ultimo exporte el dataset final raw, asi como tambien con el label encoder y estandarizado para el modelo de la notebook 2

<img src="img/capture 8.PNG" alt="Drawing"/>

<img src="img/capture 8.PNG" alt="Drawing"/>

# NoteBook 2: Modelo

Bueno, primero comento que modelo utilize y despues un poco de que cosas hubieran estado buenas probar

El plan es el siguiente:
                    

### Clustering

Primero que nada quise agregar varias features mas basadas en alguna tecnica de clustering. A que clsuter pertenecen, el centroide del clsuter, la distancia al centro del cluster, etc. Solo utilize a que clsuter pertenecian. Por ahi hubiera estado bueno haber probado con mas features. Utilize un Self-Organazing-Map para realizar el clsutering. [Paper SOM](./Review-of-SOM.pdf).

<img src="img/Capture 12.PNG" alt="Drawing"/>

<img src="img/Capture 11.PNG" alt="Drawing"/>

8 Clusters con las proporciones de churn. Entrene el SOM sin supervision, podria haberlo entrenado de forma supervisada con los datos de Evaluate. Podria ser otra cosa que probar

### Idea de modelo

Mi idea era la siguiente. Entrenar un emmbeding en los datos, prederi sobre los mismos y esa prediccion usarla para entrenar distintos arboles.
La idea era entrenar un par de modelos, y luego hacer un promedio de las probabilidades de cada modelo. Como es un promedio, si un modelo tiene pred = 1 y otro pred = 0.5, 
el de pred = 1 tiene mas peso, cosa que es bueno por que el modelo esta mas seguro. En este caso solo entrene 1 modelo pero la idea se entiende mas o menos.  

[Paper Embedding](./Entity-Embeddings-of-Categorical-Variables.pdf).

<img src="img/Capture  10.png" alt="Drawing"/>

### Embedding

<img src="img/Capture 13.png" alt="Drawing"/>

<img src="img/Capture 14.png" alt="Drawing"/>

<img src="img/Capture 15.png" alt="Drawing"/>

## LGBM CON BUSQUEDA BAYESIANA

<img src="img/Capture 16.png" alt="Drawing"/>

<img src="img/Capture 17.png" alt="Drawing"/>

<img src="img/Capture 17.5.png" alt="Drawing"/>

Hay un poco de overfitting. Tendria que bajar los estimators, bajarle al max depth o por ahi subir el learning rate y explorar un poco por ahi. \
El f2score no dio tan mal, y los datos no estan tan desbalanceados

### Analisis del modelo

Distintas metricas en base al threshold. En este problema entiendo que me gustaria meterle un poco mas al recall, ya que prefiero encontrar al que va a churn y si agarro a alguno que no se iba a ir pero mi modelo predice que si no es tan importante

<img src="img/Capture 18.png" alt="Drawing"/>

Confusion matrix con threshold en 0.5
<img src="img/Capture 19.png" alt="Drawing"/>

Logg Odds de la prediccion. El accuracy se maximiza en threshold 0.5 (es decir el 0 en el grafico)
<img src="img/Capture 20.png" alt="Drawing"/>

Probabilidades de las predicciones. Se puede observar que el modelo esta bastante seguro de las predicciones que genera. No se si es tan bueno, ya que no estar tan seguro puede 
ser mejor a la hora de generalizar. Ademas prefiero que me diga que una persona tiene 0.7% de chance de irse, que me diga 0.99%
<img src="img/Capture 21.png" alt="Drawing"/>

Finalmente un grafico de feature importance del modelo. Se puede observar que la cantidad de dinero que gastaron, el promedio de descuento y la ciudad a al que pertenecen influye mucho en la clasificacion.


<img src="img/Capture 22.png" alt="Drawing"/>

Habia un par de modelos que probar. Estuve leyendo un paper que usaba embeddings a la entrada y a la salida de una red neuronal. Otros usaban CNN para problemas de churn en problemas de retail y de e-commerce. Hay un par de ideas, pero tampoco da la data como para probar cosas tan potentes. Usar un lgbm en 45k datos ya es una salvajada, pero bueno.